In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pyodbc
from datetime import datetime

# Connect to SQL Server Database
def connect_to_db():
    try:
        conn = pyodbc.connect(
            'DRIVER={ODBC Driver 17 for SQL Server};'
            'SERVER=your_server_name;'
            'DATABASE=Library;'
            'UID=your_username;'
            'PWD=your_password;'
        )
        return conn
    except Exception as e:
        messagebox.showerror("Error", f"Failed to connect to database: {e}")
        return None

# Fetch Category and Publisher mappings
def fetch_mappings():
    try:
        conn = connect_to_db()
        if conn is None:
            return {}, {}

        cursor = conn.cursor()

        # Fetch categories
        cursor.execute("SELECT Cat_ID, Cat_Name FROM Category")
        categories = {row.Cat_Name: row.Cat_ID for row in cursor.fetchall()}

        # Fetch publishers
        cursor.execute("SELECT Pub_ID, Pub_Name FROM Publisher")
        publishers = {row.Pub_Name: row.Pub_ID for row in cursor.fetchall()}

        conn.close()
        return categories, publishers
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")
        return {}, {}

# Validate and Convert Date Format Function
def validate_and_convert_date(input_date):
    try:
        date_obj = datetime.strptime(input_date, "%d_%m_%Y")
        return date_obj.strftime("%Y-%m-%d")
    except ValueError:
        raise ValueError("Invalid date format! Please enter the date in DD_MM_YYYY format.")

# Global Variables
category_mapping, publisher_mapping = fetch_mappings()

# UI Variables
root = tk.Tk()
root.title("Library Management")
root.geometry("800x600")
root.configure(bg='#f4f4f9')

isbn_var = tk.StringVar()
title_var = tk.StringVar()
edition_var = tk.StringVar()
year_var = tk.StringVar()
price_var = tk.StringVar()
category_id_var = tk.StringVar()
publisher_id_var = tk.StringVar()

# Add Book Function
def add_book():
    try:
        # Ensure all required fields are filled
        if not (isbn_var.get() and title_var.get() and year_var.get() and price_var.get() and category_id_var.get() and publisher_id_var.get()):
            raise ValueError("All fields must be filled!")

        # Further validate the price (it should be a positive number)
        try:
            price = float(price_var.get())
            if price <= 0:
                raise ValueError("Price must be a positive value.")
        except ValueError:
            raise ValueError("Invalid price! Please enter a valid number greater than 0.")

        # Convert and validate the date
        sql_date = validate_and_convert_date(year_var.get())

        conn = connect_to_db()
        if conn is None:
            return

        cursor = conn.cursor()

        query = """
            INSERT INTO books (ISBN, Title, Edition, Year_of_publication, Price, Cat_ID, Pub_ID)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        cursor.execute(
            query,
            isbn_var.get(),
            title_var.get(),
            edition_var.get(),
            sql_date,
            price_var.get(),
            category_mapping[category_id_var.get()],
            publisher_mapping[publisher_id_var.get()]
        )
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Book added successfully!")
        fetch_books()
    except ValueError as ve:
        messagebox.showerror("Error", str(ve))
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Update Book Function
def update_book():
    try:
        conn = connect_to_db()
        if conn is None:
            return

        cursor = conn.cursor()

        # Convert and validate the date
        sql_date = validate_and_convert_date(year_var.get())

        query = """
            UPDATE books
            SET Title = ?, Edition = ?, Year_of_publication = ?, Price = ?, Cat_ID = ?, Pub_ID = ?
            WHERE ISBN = ?
        """
        cursor.execute(
            query,
            title_var.get(),
            edition_var.get(),
            sql_date,
            price_var.get(),
            category_mapping[category_id_var.get()],
            publisher_mapping[publisher_id_var.get()],
            isbn_var.get()
        )
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Book updated successfully!")
        fetch_books()
    except ValueError as ve:
        messagebox.showerror("Error", str(ve))
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Delete Book Function (Updated)
def delete_book_by_title_or_year():
    try:
        # Get input values (you could use additional UI fields for criteria selection)
        title_to_delete = title_var.get()  # Assume you have a title field to delete by
        year_to_delete = year_var.get()   # Assume you also have a year field for deletion criteria

        if not title_to_delete and not year_to_delete:
            messagebox.showwarning("Warning", "Please enter title or year to delete a book!")
            return

        # Connect to the database
        conn = connect_to_db()
        if conn is None:
            return

        cursor = conn.cursor()

        # Construct DELETE query (based on title or year)
        if title_to_delete:
            cursor.execute("DELETE FROM books WHERE Title = ?", title_to_delete)
        elif year_to_delete:
            cursor.execute("DELETE FROM books WHERE Year_of_publication = ?", year_to_delete)

        conn.commit()
        conn.close()

        messagebox.showinfo("Success", "Books deleted successfully!")
        fetch_books()

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Fetch Books Function
def fetch_books():
    try:
        conn = connect_to_db()
        if conn is None:
            return

        cursor = conn.cursor()
        query = """
            SELECT b.ISBN, b.Title, b.Edition, b.Year_of_publication, b.Price,
                   c.Cat_Name, p.Pub_Name
            FROM books b
            JOIN Category c ON b.Cat_ID = c.Cat_ID
            JOIN Publisher p ON b.Pub_ID = p.Pub_ID
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()

        for row in book_table.get_children():
            book_table.delete(row)

        for row in rows:
            formatted_row = tuple(str(value) for value in row)
            book_table.insert("", tk.END, values=formatted_row)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Populate Form for Update
def populate_form(event):
    selected = book_table.item(book_table.selection())['values']
    if selected:
        isbn_var.set(selected[0])
        title_var.set(selected[1])
        edition_var.set(selected[2])
        year_var.set(selected[3])
        price_var.set(selected[4])
        category_id_var.set(selected[5])
        publisher_id_var.set(selected[6])

# UI Layout
form_frame = tk.Frame(root, bg='#f4f4f9')
form_frame.pack(side=tk.TOP, fill=tk.X, padx=20, pady=10)

tk.Label(form_frame, text="ISBN", bg='#f4f4f9').grid(row=0, column=0, padx=5, pady=5)
tk.Entry(form_frame, textvariable=isbn_var).grid(row=0, column=1, padx=5, pady=5)

tk.Label(form_frame, text="Title", bg='#f4f4f9').grid(row=1, column=0, padx=5, pady=5)
tk.Entry(form_frame, textvariable=title_var).grid(row=1, column=1, padx=5, pady=5)

tk.Label(form_frame, text="Edition", bg='#f4f4f9').grid(row=2, column=0, padx=5, pady=5)
tk.Entry(form_frame, textvariable=edition_var).grid(row=2, column=1, padx=5, pady=5)

tk.Label(form_frame, text="Year (DD_MM_YYYY)", bg='#f4f4f9').grid(row=3, column=0, padx=5, pady=5)
tk.Entry(form_frame, textvariable=year_var).grid(row=3, column=1, padx=5, pady=5)

tk.Label(form_frame, text="Price", bg='#f4f4f9').grid(row=4, column=0, padx=5, pady=5)
tk.Entry(form_frame, textvariable=price_var).grid(row=4, column=1, padx=5, pady=5)

tk.Label(form_frame, text="Category", bg='#f4f4f9').grid(row=5, column=0, padx=5, pady=5)
tk.OptionMenu(form_frame, category_id_var, *category_mapping.keys()).grid(row=5, column=1, padx=5, pady=5)

tk.Label(form_frame, text="Publisher", bg='#f4f4f9').grid(row=6, column=0, padx=5, pady=5)
tk.OptionMenu(form_frame, publisher_id_var, *publisher_mapping.keys()).grid(row=6, column=1, padx=5, pady=5)

button_frame = tk.Frame(root, bg='#f4f4f9')
button_frame.pack(side=tk.TOP, fill=tk.X, padx=20, pady=10)

tk.Button(button_frame, text="Add Book", command=add_book, bg="green", fg="white").pack(side=tk.LEFT, padx=5)
tk.Button(button_frame, text="Update Book", command=update_book, bg="orange", fg="white").pack(side=tk.LEFT, padx=5)
tk.Button(button_frame, text="Delete Book by Title/Year", command=delete_book_by_title_or_year, bg="red", fg="white").pack(side=tk.LEFT, padx=5)

table_frame = tk.Frame(root, bg='#f4f4f9')
table_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

columns = ("ISBN", "Title", "Edition", "Year", "Price", "Category", "Publisher")
book_table = ttk.Treeview(table_frame, columns=columns, show="headings")
for col in columns:
    book_table.heading(col, text=col)
    book_table.column(col, minwidth=50, width=120, stretch=False, anchor="center")
book_table.pack(fill=tk.BOTH, expand=True)
book_table.bind("<ButtonRelease-1>", populate_form)

fetch_books()
root.mainloop()
